In [4]:
from __future__ import print_function
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
import numpy as np

In [6]:
X = np.random.randn(10000,2)
Y = 2 * X[:,0] - 3.4 * X[:,1] + 4.2 + .01 * np.random.normal(size=10000)

ctx = mx.cpu()
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))
net.collect_params().initialize()
loss = gluon.loss.L2Loss()

In [7]:
batch_sizes = [1,10,100,1000,10000]
learning_rates = [0.1,0.1,0.5,0.5,1.0]
epochs = 3

In [8]:
for batch_size, learning_rate in zip(batch_sizes, learning_rates):
    trainer = gluon.Trainer(net.collect_params(), 'sgd',
                            {'learning_rate': learning_rate})
    net.collect_params().initialize(mx.init.Xavier(magnitude=2.24),
                                    ctx=ctx, force_reinit=True)
    train_data = mx.io.NDArrayIter(X, Y, batch_size=batch_size, shuffle=True)

    for e in range(epochs):
        train_data.reset()
        for i, batch in enumerate(train_data):
            data = batch.data[0].as_in_context(ctx)
            label = batch.label[0].as_in_context(ctx).reshape((-1, 1))
            with autograd.record():
                output = net(data)
                mse = loss(output, label)
            mse.backward()
            trainer.step(data.shape[0])

    for para_name, para_value in net.collect_params().items():
        print("Batch size:", batch_size, para_name,
              para_value.data().asnumpy()[0])

Batch size: 1 dense0_weight [ 1.9985865 -3.402167 ]
Batch size: 1 dense0_bias 4.197694
Batch size: 10 dense0_weight [ 1.9999421 -3.3997474]
Batch size: 10 dense0_bias 4.1982126
Batch size: 100 dense0_weight [ 1.9990214 -3.400498 ]
Batch size: 100 dense0_bias 4.19952
Batch size: 1000 dense0_weight [ 1.9996382 -3.4000344]
Batch size: 1000 dense0_bias 4.199864
Batch size: 10000 dense0_weight [ 1.9999006 -3.3999245]
Batch size: 10000 dense0_bias 4.199973
